In [1]:
# !pip install aix360

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from tqdm import tqdm_notebook
import torch

from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
from aix360.algorithms.protodash import ProtodashExplainer
from aix360.datasets.heloc_dataset import HELOCDataset

Using TensorFlow backend.


In [3]:
def load_data(path, target):
    df = pd.read_csv(path)
    df_origin = df.copy()
    
    num_cols = []
    cat_cols = []

    for col in df.columns:
        if col in [target]:
            continue
        if (df[col].dtype == 'object'):
            cat_cols.append(col)
        else:
            num_cols.append(col)
    
    for col in num_cols:
        scaler = StandardScaler()
        scaler.fit(df[col].values.reshape(-1, 1))

        df[col] = scaler.transform(df[col].values.reshape(-1, 1))
    
    
    for f in cat_cols:
        lbl = preprocessing.LabelEncoder()
        print(f, df[f].nunique())
        lbl.fit(df[f])
        df[f] = lbl.transform(list(df[f].astype(str)))
    
    not_used = [target]
    used_features = [x for x in df.columns if x not in not_used]
    X = df[used_features]
    y = df[target]
    
    X = torch.tensor(np.array(X), dtype=torch.float)
    y = torch.tensor(np.array(y), dtype=torch.long)
    
    return X, y, df_origin

In [4]:
path = '../../demo_data/adult.csv'
target = 'income'

X, y, df = load_data(path, target)

workclass 9
education 16
marital.status 7
occupation 15
relationship 6
race 5
sex 2
native.country 42


In [5]:
X.shape, y.shape, df.shape

(torch.Size([32561, 14]), torch.Size([32561]), (32561, 15))

In [6]:
used = [x for x in df.columns if x != target]
from sklearn.model_selection import train_test_split

xn_train, xn_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # normalized data
x_train, x_test = train_test_split(df[used], test_size=0.2, random_state=42) 
x_train.index = range(len(x_train))
x_test.index = range(len(x_test))

In [7]:
# nn with no softmax
def nn_small():
    model = Sequential()
    model.add(Dense(10, input_dim=len(used), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  
    return model

In [8]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['income<50', 'incode>=50']

# compile and print model summary
nn = nn_small()
nn.summary()

# train model or load a trained model
TRAIN_MODEL = False

nn.fit(xn_train, y_train, batch_size=128, epochs=10, verbose=1, shuffle=False)
        
# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                150       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________

Epoch 1/10
26048/26048 [==============================] - 0s 13us/step - loss: 0.8865 - accuracy: 0.7448
Epoch 2/10
26048/26048 [==============================] - 0s 10us/step - loss: 0.4324 - accuracy: 0.7955
Epoch 3/10
26048/26048 [==============================] - 0s 10us/step - loss: 0.4041 - accuracy: 0.8158
Epoch 4/10
26048/26048 [==============================] - 0s 10us/step - loss: 0.3963 - accuracy: 0.8201
Epo

In [9]:
p_train = nn.predict_classes(xn_train) # Use trained neural network to predict train points
p_train = p_train.reshape((p_train.shape[0],1))

z_train = np.hstack((xn_train, p_train)) # Store (normalized) instances that were predicted as Good
z_train_good = z_train[z_train[:,-1]==1, :]

zun_train = np.hstack((x_train, p_train)) # Store (unnormalized) instances that were predicted as Good 
zun_train_good = zun_train[zun_train[:,-1]==1, :]

Obtain similar samples as explanations(for a example income>=50) 

In [10]:
idx = 39

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)

print("Chosen Sample:", idx)
if nn.predict_proba(X) <= 0.5:
  print("Prediction made by the model:", class_names[0])
else:
  print("Prediction made by the model:", class_names[1])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

# attach the prediction made by the model to X
X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

Chosen Sample: 39
Prediction made by the model: incode>=50
Prediction probabilities: [[0.87907875]]



In [11]:
x_test.iloc[idx]

age                               39
workclass               Self-emp-inc
fnlwgt                        283338
education                  Bachelors
education.num                     13
marital.status    Married-civ-spouse
occupation           Exec-managerial
relationship                 Husband
race                           White
sex                             Male
capital.gain                    7298
capital.loss                       0
hours.per.week                    40
native.country         United-States
Name: 39, dtype: object

In [12]:
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X, z_train_good, m=5)

In [13]:
dfs = pd.DataFrame.from_records(zun_train_good[S, 0:-1])
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_good[S[i], -1])]) # Append class names

In [14]:
dfs[dfs.shape[1]+1] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Calculate normalized importance weights
dfs.transpose()

,0,1,2,3,4
age,41,35,31,42,21
workclass,Self-emp-inc,Private,Private,State-gov,Private
fnlwgt,220821,1226583,1033222,222884,34310
education,Bachelors,Bachelors,Bachelors,Bachelors,Assoc-voc
education.num,13,13,13,13,11
marital.status,Married-civ-spouse,Married-civ-spouse,Never-married,Divorced,Married-civ-spouse
occupation,Exec-managerial,Sales,Adm-clerical,Adm-clerical,Craft-repair
relationship,Husband,Husband,Not-in-family,Not-in-family,Husband
race,White,White,White,White,White
sex,Male,Male,Male,Male,Male


In [15]:
z = z_train_good[S, 0:-1] # Store chosen prototypes
eps = 1e-10 # Small constant defined to eliminate divide-by-zero errors
fwt = np.zeros(z.shape)
for i in range (z.shape[0]):
    for j in range(z.shape[1]):
        fwt[i, j] = np.exp(-1 * abs(X[0, j] - z[i,j])/(np.std(z[:, j])+eps)) # Compute feature similarity in [0,1]
                
# move wts to a dataframe to display
dfw = pd.DataFrame.from_records(np.around(fwt.astype('double'), 2))
dfw.columns = df.columns[:-1]
dfw.transpose()

,0,1,2,3,4
age,0.77,0.59,0.35,0.68,0.09
workclass,1.00,0.42,0.42,0.18,0.42
fnlwgt,0.88,0.14,0.21,0.88,0.60
education,1.00,1.00,1.00,1.00,0.08
education.num,1.00,1.00,1.00,1.00,0.08
marital.status,1.00,1.00,0.21,0.21,1.00
occupation,1.00,0.14,0.48,0.48,0.78
relationship,1.00,1.00,0.13,0.13,1.00
race,1.00,1.00,1.00,1.00,1.00
sex,1.00,1.00,1.00,1.00,1.00


In [16]:
z_train_bad = z_train[z_train[:,-1]==0, :]
zun_train_bad = zun_train[zun_train[:,-1]==0, :]

Obtain similar samples as explanations(for a example income<50) 

In [17]:
idx = 8

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Chosen Sample:", idx)
if nn.predict_proba(X) <= 0.5:
  print("Prediction made by the model:", class_names[0])
else:
  print("Prediction made by the model:", class_names[1])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

Chosen Sample: 8
Prediction made by the model: income<50
Prediction probabilities: [[0.14873022]]



In [18]:
x_test.iloc[idx]

age                               28
workclass                    Private
fnlwgt                        166481
education                    7th-8th
education.num                      4
marital.status    Married-civ-spouse
occupation         Handlers-cleaners
relationship                 Husband
race                           Other
sex                             Male
capital.gain                       0
capital.loss                    2179
hours.per.week                    40
native.country           Puerto-Rico
Name: 8, dtype: object

In [19]:
(W, S, setValues) = explainer.explain(X, z_train_bad, m=5) # Return weights W, Prototypes S and objective function values

In [20]:
# move samples to a dataframe to display
dfs = pd.DataFrame.from_records(zun_train_bad[S, 0:-1])
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_bad[S[i], -1])]) # Append class names
dfs[dfs.shape[1]+1] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Compute normalized importance weights for prototypes
dfs.transpose()

,0,1,2,3,4
age,29,35,37,22,46
workclass,Private,Private,Self-emp-inc,Private,Private
fnlwgt,202878,163392,107164,119592,147640
education,7th-8th,Some-college,10th,Assoc-acdm,5th-6th
education.num,4,10,6,12,3
marital.status,Married-civ-spouse,Married-civ-spouse,Never-married,Never-married,Married-civ-spouse
occupation,Farming-fishing,Transport-moving,Transport-moving,Handlers-cleaners,Transport-moving
relationship,Husband,Husband,Not-in-family,Not-in-family,Husband
race,White,Asian-Pac-Islander,White,Black,Amer-Indian-Eskimo
sex,Male,Male,Male,Male,Male


In [21]:
z = z_train_bad[S, 0:-1] # Store the prototypes
eps = 1e-10 # Small constant to guard against divide by zero errors
fwt = np.zeros(z.shape)
for i in range (z.shape[0]): # Compute feature similarity for each prototype
    for j in range(z.shape[1]):
        fwt[i, j] = np.exp(-1 * abs(X[0, j] - z[i,j])/(np.std(z[:, j])+eps))
                
# move wts to a dataframe to display
dfw = pd.DataFrame.from_records(np.around(fwt.astype('double'), 2))
dfw.columns = df.columns[:-1]
dfw.transpose()

,0,1,2,3,4
age,0.88,0.42,0.33,0.47,0.11
workclass,1.00,1.00,0.08,1.00,1.00
fnlwgt,0.34,0.91,0.17,0.25,0.57
education,1.00,0.13,0.36,0.67,0.82
education.num,1.00,0.18,0.56,0.10,0.75
marital.status,1.00,1.00,0.13,0.13,1.00
occupation,0.79,0.15,0.15,1.00,0.15
relationship,1.00,1.00,0.13,0.13,1.00
race,0.54,0.29,0.54,0.54,0.15
sex,1.00,1.00,1.00,1.00,1.00


ref: [HELOC.ipynb](https://github.com/Trusted-AI/AIX360/blob/master/examples/tutorials/HELOC.ipynb)